# Project Work Part 1

#### a)
Downloading data and displaying some of it

In [2]:
import numpy as np
import polars as pl

# read data
links_df = pl.read_csv("../data/links.csv", schema={"movieId": pl.Int32, "imdb": pl.Int32, "tmdbId": pl.Int32})
movies_df = pl.read_csv("../data/movies.csv", schema={"movieId": pl.Int32, "title": pl.String, "genres": pl.String})
ratings_df = pl.read_csv("../data/ratings.csv", schema={"userId": pl.Int32, "movieId": pl.Int32, "rating": pl.Float32, "timestamp": pl.Int32})
tags_df = pl.read_csv("../data/tags.csv", schema={"userId": pl.Int32, "movieId": pl.Int32, "tag": pl.String, "timestamp": pl.Int32})

print("Number of ratings: ", ratings_df.height)

Number of ratings:  100836
Number of ratings:  100836


In [226]:
print(links_df.head())
print('rows: ', links_df.height)

shape: (5, 3)
┌─────────┬────────┬────────┐
│ movieId ┆ imdb   ┆ tmdbId │
│ ---     ┆ ---    ┆ ---    │
│ i32     ┆ i32    ┆ i32    │
╞═════════╪════════╪════════╡
│ 1       ┆ 114709 ┆ 862    │
│ 2       ┆ 113497 ┆ 8844   │
│ 3       ┆ 113228 ┆ 15602  │
│ 4       ┆ 114885 ┆ 31357  │
│ 5       ┆ 113041 ┆ 11862  │
└─────────┴────────┴────────┘
rows:  9742


In [227]:
print(movies_df.head())
print('rows: ', movies_df.height)

shape: (5, 3)
┌─────────┬─────────────────────────────────┬─────────────────────────────────┐
│ movieId ┆ title                           ┆ genres                          │
│ ---     ┆ ---                             ┆ ---                             │
│ i32     ┆ str                             ┆ str                             │
╞═════════╪═════════════════════════════════╪═════════════════════════════════╡
│ 1       ┆ Toy Story (1995)                ┆ Adventure|Animation|Children|C… │
│ 2       ┆ Jumanji (1995)                  ┆ Adventure|Children|Fantasy      │
│ 3       ┆ Grumpier Old Men (1995)         ┆ Comedy|Romance                  │
│ 4       ┆ Waiting to Exhale (1995)        ┆ Comedy|Drama|Romance            │
│ 5       ┆ Father of the Bride Part II (1… ┆ Comedy                          │
└─────────┴─────────────────────────────────┴─────────────────────────────────┘
rows:  9742


In [228]:
print(ratings_df.head())
print('rows: ', ratings_df.height)

shape: (5, 4)
┌────────┬─────────┬────────┬───────────┐
│ userId ┆ movieId ┆ rating ┆ timestamp │
│ ---    ┆ ---     ┆ ---    ┆ ---       │
│ i32    ┆ i32     ┆ f32    ┆ i32       │
╞════════╪═════════╪════════╪═══════════╡
│ 1      ┆ 1       ┆ 4.0    ┆ 964982703 │
│ 1      ┆ 3       ┆ 4.0    ┆ 964981247 │
│ 1      ┆ 6       ┆ 4.0    ┆ 964982224 │
│ 1      ┆ 47      ┆ 5.0    ┆ 964983815 │
│ 1      ┆ 50      ┆ 5.0    ┆ 964982931 │
└────────┴─────────┴────────┴───────────┘
rows:  100836


In [229]:
print(tags_df.head())
print('rows: ', tags_df.height)

shape: (5, 4)
┌────────┬─────────┬─────────────────┬────────────┐
│ userId ┆ movieId ┆ tag             ┆ timestamp  │
│ ---    ┆ ---     ┆ ---             ┆ ---        │
│ i32    ┆ i32     ┆ str             ┆ i32        │
╞════════╪═════════╪═════════════════╪════════════╡
│ 2      ┆ 60756   ┆ funny           ┆ 1445714994 │
│ 2      ┆ 60756   ┆ Highly quotable ┆ 1445714996 │
│ 2      ┆ 60756   ┆ will ferrell    ┆ 1445714992 │
│ 2      ┆ 89774   ┆ Boxing story    ┆ 1445715207 │
│ 2      ┆ 89774   ┆ MMA             ┆ 1445715200 │
└────────┴─────────┴─────────────────┴────────────┘
rows:  3683


#### b)
User-based collaborative filtering using the Pearson correlation function

In [152]:
# Create a ratings matrix by pivoting the ratings dataframe, filling null values with 0
ratings_matrix = (
    ratings_df
    .drop("timestamp")
    .sort("movieId")
    .pivot("movieId", index="userId")
    .sort("userId")
    .fill_null(strategy="zero")
)
ratings_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,…,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
# Combine movie rating columns into rating vector
rating_vectors = (
    ratings_matrix
    .with_columns(rating_vector=pl.concat_list(ratings_matrix.columns[1:]))
    .drop(pl.exclude(["userId", "rating_vector"]))
)
rating_vectors.head()

userId,rating_vector
i32,list[f32]
1,"[4.0, 0.0, … 0.0]"
2,"[0.0, 0.0, … 0.0]"
3,"[0.0, 0.0, … 0.0]"
4,"[0.0, 0.0, … 0.0]"
5,"[4.0, 0.0, … 0.0]"


In [252]:
# Count similarity scores for userId 1
query_user = rating_vectors.filter(pl.col("userId") == 1).to_numpy()[0][1]

# The pearson correlation function is found in scipy package and can be applied into dataframe directly
import scipy as sp
rating_vectors = (
    rating_vectors
    .with_columns(
     pl.col("rating_vector")
     .map_elements(
         lambda x: sp.stats.pearsonr(query_user, x)[0], return_dtype=pl.Float32
                   ).alias("similarity_score")
    )
    .sort("similarity_score", descending=True)
)

# take top 50 most similar users to userId 1
top_50_most_similar = rating_vectors.filter(pl.col("userId") != 1).limit(50)
top_50_most_similar

userId,rating_vector,similarity_score
i32,list[f32],f32
266,"[2.0, 0.0, … 0.0]",0.344983
313,"[0.0, 0.0, … 0.0]",0.333876
368,"[0.0, 0.0, … 0.0]",0.324041
57,"[5.0, 0.0, … 0.0]",0.323948
39,"[0.0, 0.0, … 0.0]",0.32012
…,…,…
307,"[4.0, 2.5, … 0.0]",0.239943
428,"[0.0, 0.0, … 0.0]",0.237911
453,"[5.0, 0.0, … 0.0]",0.237317


#### b)
Prediction function

In [278]:
# Find candidates for recommendation by selecting movies that the similar users have rated

query_user_rated_movie_ids = np.nonzero(query_user)[0] + 1

similar_user_rated_movie_ids = (
    top_50_most_similar
    .with_columns(
        pl.col("rating_vector")
        .map_elements(
             lambda x: np.nonzero(x)[0] + 1, return_dtype=pl.List(pl.Int32) # Calculate non zero indices and add 1 to make it movieId
                       )
        .alias("movieId")
    )
)

similar_users_most_watched_movies = (
    similar_user_rated_movie_ids
    .select("movieId")
    .explode("movieId")
    .group_by("movieId")
    .agg(
        pl.count("movieId")
        .alias("count")
    )
    .sort("count", descending=True)
    .filter(pl.col("movieId").is_in(query_user_rated_movie_ids).not_())
    .filter(pl.col("count") > 20)) # At least 20 users must have rated the movie

print(similar_users_most_watched_movies)

candidate_movie_ids = similar_users_most_watched_movies.select("movieId").to_numpy().flatten()

shape: (119, 2)
┌─────────┬───────┐
│ movieId ┆ count │
│ ---     ┆ ---   │
│ i32     ┆ u32   │
╞═════════╪═══════╡
│ 508     ┆ 46    │
│ 794     ┆ 44    │
│ 660     ┆ 41    │
│ 902     ┆ 40    │
│ 2078    ┆ 40    │
│ …       ┆ …     │
│ 4927    ┆ 21    │
│ 1009    ┆ 21    │
│ 2325    ┆ 21    │
│ 3007    ┆ 21    │
│ 2593    ┆ 21    │
└─────────┴───────┘


In [279]:
# TODO: Implement prediction function by doing some nice matrix operations

def predict_movies(user_ratings, similar_users_ratings, similarity_scores, candidate_movie_ids):
    """

    :param user_ratings: sparse numpy array of user ratings
    :param similar_users_ratings: sparse numpy 2D matrix of similar users ratings. Each row is a user and each column is a movie
    :param similarity_scores: numpy array of similarity scores between the user and the similar users
    :param candidate_movie_ids: numpy array of movie ids that the similar users have rated
    :return: ranked list of candidate movie ids
    """
    mean_a = np.mean(user_ratings)



